# ML. Assignment 3. Task 3
## Oleh Lukianykhin, Yevhen Pozdniakov

## a. Joint distribution of $(x^{pt},y^{pt},z^{pt})$

1. As $x^{pt}=y^{pt} + z^{pt} + \epsilon$, i.e. is a sum of normaly distributed random variables, it is normally distributed as well. Hence, $x^{pt}\sim N(\mu_x;\sigma_x)$

To find parameters of this normal distribution properties of expectation and dispersion will be used:
$$E[x^{pt}] = E[y^{pt}] + E[z^{pt}] + E[\epsilon] = \mu_p + \nu_t,$$
$$D[x^{pt}] = D[y^{pt}] + D[z^{pt}] + D[\epsilon] +cov[y^{pt};z^{pt}] +cov[y^{pt};\epsilon] +cov[z^{pt};\epsilon]$$

As $y^{pt}, z^{pt}, \epsilon$ are independent, their covariances equal zero. Thus:
$$D[x^{pt}] = \sigma^2_p+\tau^2_t+\sigma^2$$


2. Joint distribution is normal $N(\mu;\Sigma)$. Parameters can be determined as follows:
$$\mu=\begin{pmatrix} \mu_p \\ \nu_t \\ \mu_p + \nu_t\end{pmatrix}$$

For the covariance matrix $\Sigma$ one need to find covariances between elements of the joint random variable. As it was already mentioned, $cov[y^{pt};z^{pt}]=0$ as these are independent.

Using properties of covariance:
$$cov[y^{pt};x^{pt}]=cov[y^{pt};y^{pt} + z^{pt} + \epsilon]=cov[y^{pt};y^{pt}]+cov[y^{pt};z^{pt}]+cov[y^{pt};\epsilon]=D[y^{pt};y^{pt}]=\sigma^2_p$$
The same way one can derive:
$$cov[z^{pt};x^{pt}]=\tau^2_t$$

3. Thus:
$$\Sigma=\begin{pmatrix} \sigma^2_p & 0 & \sigma^2_p \\
0 & \tau^2_t & \tau^2_t \\ \sigma^2_p & \tau^2_t & \sigma^2_p+\tau^2_t+\sigma^2 \end{pmatrix}$$


## b. E-step of EM-algorithm

1. To find the proper parameters for the distributions, let's call set of these parameters $\theta$, log-likelihood function should be maximized for the given training data:
$$l=\sum_{\substack{i=1\dots P,\\ j=1\dots T}} log( p(x^{ij};\theta)),$$

However, as we have latent variables in this equation, that are not observed:
$$l=\sum_{\substack{i=1\dots P,\\ j=1\dots T}} log\left(\int_{z_{min}}^{z_{max}}\int_{y_{min}}^{y_{max}} p(x^{ij};y;z;\theta)dydz \right),$$
we can not optimize it directly.

Here integration instead of summation was used, as marks are continuous in certain defined interval. For the case of known discrete marks this can be reversed. This doesn't change the general workflow.   

2. Thus we introduce new distribution function $Q^{ij}(y;z)$:
$$l=\sum_{\substack{i=1\dots P,\\ j=1\dots T}} log\left(\int_{z_{min}}^{z_{max}}\int_{y_{min}}^{y_{max}} Q^{ij}(y;z)\cdot \frac{p(x^{ij};y;z;\theta) }{Q^{ij}(y;z)}dydz\right)\geq \\ \geq 
\sum_{\substack{i=1\dots P,\\ j=1\dots T}} \int_{z_{min}}^{z_{max}}\int_{y_{min}}^{y_{max}} Q^{ij}(y;z)\cdot log \left(\frac{p(x^{ij};y;z;\theta) }{Q^{ij}(y;z)}\right)dydz,$$

On the E-step we define $Q^{ij}(y;z) = p(y;z|x^{ij})$ to work with the lower bound.



3. According to the properties of multivariate normal distribution, conditional distribution can be found, when parameters of the original distribution are known ([formulas can be found here](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Conditional_distributions)). This is our case. 

For $Q^{ij}(y;z) = p(y;z|x^{ij})\sim N(\mu_{cond};\Sigma_{cond})$:
$$\mu_{cond}=\begin{pmatrix} \mu_p \\ \nu_t \end{pmatrix} + \begin{pmatrix} \sigma^2_p \\ \tau^2_t \end{pmatrix}\cdot(\sigma^2_p+\tau^2_t+\sigma^2)^{-1}\cdot (x^{ij}-\mu_p - \nu_t)$$


$$\mu_{cond}=\begin{pmatrix} \mu_p +\sigma^2_p\frac{x^{ij}-\mu_p - \nu_t}{\sigma^2_p+\tau^2_t+\sigma^2} \\ \nu_t +  \tau^2_t\frac{x^{ij}-\mu_p - \nu_t}{\sigma^2_p+\tau^2_t+\sigma^2}\end{pmatrix} $$

Covariance matrix is derived as follows:
$$\Sigma_{cond}=\begin{pmatrix} \sigma^2_p & 0 \\
0 & \tau^2_t \end{pmatrix} - \begin{pmatrix} \sigma^2_p \\ \tau^2_t \end{pmatrix}\cdot(\sigma^2_p+\tau^2_t+\sigma^2)^{-1} \begin{pmatrix} \sigma^2_p & \tau^2_t \end{pmatrix}$$

$$\Sigma_{cond}=\begin{pmatrix} \sigma^2_p(1-\frac{\sigma^2_p}{\sigma^2_p+\tau^2_t+\sigma^2}) & -\frac{\sigma^2_p\tau^2_t}{\sigma^2_p+\tau^2_t+\sigma^2} \\
-\frac{\sigma^2_p\tau^2_t}{\sigma^2_p+\tau^2_t+\sigma^2} & \tau^2_t(1-\frac{\tau^2_t}{\sigma^2_p+\tau^2_t+\sigma^2}) \end{pmatrix} $$

Then, $Q^{ij}(y,z)=p(y;z|x^{ij})\sim N(\mu_{cond};\Sigma_{cond})$
$$Q^{ij}(y,z)= \frac{1}{|\Sigma_{cond}|^{1/2} (2\pi) } e^{-1/2\left( (y;z)^T - \mu_{cond})^T\Sigma_{cond}^{-1} (y;z)^T - \mu_{cond}) \right)}$$

-------------------------

Alternatively, there is a simpler way to determine $p(y;z|x^{ij})$  - Bayes rule:
$$p(y;z|x^{ij})=\frac{p(y;z;x^{ij})}{p(x^{ij})}$$

As we know both numerator and denominator, it is possible to derive the required distribution function:
$$p(y;z|x^{ij})=\frac{\frac{1}{|\Sigma|^{1/2} (2\pi)^{3/2} } e^{-1/2\left( (y;z;x^{ij})^T - \mu)^T\Sigma^{-1} (y;z;x^{ij})^T - \mu) \right)}}
{\frac{1}{\sigma \sqrt {2\pi } } e^{-\left( x^{ij} - \mu_x  \right)^2/2\sigma_x^2 } }  $$

Thus both formulas can be used for setting $Q^{ij}(y;z)$ on the E-step.

## c. M-step of EM-algorithm

So, on the Maximization step of the algorithm the following expression will be maximized with respect to $\theta=[\mu_p,\nu_t,\sigma^2_p,\tau^2_t]$:
$$l_{\text{lower bound}} = 
\sum_{\substack{i=1\dots P,\\ j=1\dots T}} \int_{z_{min}}^{z_{max}}\int_{y_{min}}^{y_{max}} Q^{ij}(y;z)\cdot log \left(\frac{p(x^{ij};y;z;\theta) }{Q^{ij}(y;z)}\right)dydz,$$

Or M-step can be written as follows:
$$\theta=[\mu_p,\nu_t,\sigma^2_p,\tau^2_t] = \underset{\theta}{\operatorname{argmax}} l_{\text{lower bound}} = \underset{\theta}{\operatorname{argmax}}\left(
\sum_{\substack{i=1\dots P,\\ j=1\dots T}} \int_{z_{min}}^{z_{max}}\int_{y_{min}}^{y_{max}} Q^{ij}(y;z)\cdot log \left(\frac{p(x^{ij};y;z;\theta) }{Q^{ij}(y;z)}\right)dydz \right),$$

### Again, integral are used for the purpose of approach generalization. If in this case project marks are discrete from the given interval it is possible to replace intagration with the summation and consequently simplify the process.